# Create custom federated architectures using FLEXible

In this notebook we show how to create customs architetures when working FLEXible. We hope that this notebook will help users to learn how to use the `FlexPool` class in case they want to create different architectures from the two available in the `FlexPool` class. Those methods are, `client_server_architecture` and `p2p_architecture`, and are the most probable type of architectures that might be found during a federated learning experiment, but it might be interesting to create another architecture.

In [ ]:
from flex.pool import FlexPool
from flex.actors import FlexActors, FlexRole
from flex.data import FedDataDistribution, Dataset

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In this example we are going to use a simple dataset. This dataset is the Iris dataset, available in the sklearn library. Then, we are going to federate it as we need to have a Federated Dataset to create customs architectures using `FlexPool`.

In [ ]:
iris = load_iris()
# Generate train-test splits
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

train_iris = Dataset.from_numpy(X_train, y_train)
test_iris = Dataset.from_numpy(X_test, y_test)

federated_iris = FedDataDistribution.iid_distribution(train_iris, n_clients=5)

For each custom architecture, we're going to use the client's ids. As we didn't specify any custom client's ids, our ids are going to be numerical from 1 to N_clients.

Now it's time to create some functions to create custom architectures.

In [ ]:
def custom_architecture_clients_as_aggregators(node_ids: list) -> FlexActors :
    if "server" in node_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    actors = FlexActors()

    for clitent_id in node_ids:
        actors[clitent_id] = FlexRole.aggregator_client

    actors['server'] = FlexRole.server

    return actors

In [ ]:
clients_aggregators_actors = custom_architecture_clients_as_aggregators(list(federated_iris.keys()))

clients_aggregators_pool = FlexPool(federated_iris, clients_aggregators_actors)

clients = clients_aggregators_pool.clients
aggregators = clients_aggregators_pool.aggregators
servers = clients_aggregators_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

print(f"Clients and aggregators are the same actors: {sorted(clients.actor_ids) == sorted(aggregators.actor_ids)}")

Now lets create an architecture where a number of clients will act as aggregators too.

In [ ]:
def custom_architecture_n_clients_as_aggregators(node_ids: list, n_clients_as_aggregators) -> FlexActors :
    if "server" in node_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    n_clients_as_aggregators = min(n_clients_as_aggregators, len(node_ids))

    actors = FlexActors()

    for client_id in range(n_clients_as_aggregators):
        actors[client_id] = FlexRole.aggregator_client

    for client_id in range(n_clients_as_aggregators, len(node_ids)):
        actors[client_id] = FlexRole.client

    actors['server'] = FlexRole.server

    return actors

In [ ]:
n_clients_as_aggregators_actors = custom_architecture_n_clients_as_aggregators(list(federated_iris.keys()), n_clients_as_aggregators=5)

n_clients_as_aggregators_pool = FlexPool(federated_iris, n_clients_as_aggregators_actors)

clients = n_clients_as_aggregators_pool.clients
aggregators = n_clients_as_aggregators_pool.aggregators
servers = n_clients_as_aggregators_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

Now lets create a "random" custom architecture, that will be moreover manually created.

In [ ]:
def custom_architecture(node_ids: list) -> FlexActors :
    if "server" in node_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    import numpy as np

    n_nodes = len(node_ids)
    print(f"Total number of available clients: {n_nodes}")
    n_clients_aggregators = np.random.randint(low=0, high=n_nodes//2)
    print(f"Clients as aggregators: {n_clients_aggregators}")
    n_clients_servers = np.random.randint(low=0, high=n_nodes-n_clients_aggregators)
    print(f"Clients as servers: {n_clients_servers}")
    rest_clients = n_nodes - (n_clients_aggregators + n_clients_servers)
    print(f"Only clients: {rest_clients}")
    actors = FlexActors(
        {node_ids[client_id]: FlexRole.aggregator_client for client_id in range(n_clients_aggregators)}
    )

    actors.update(
        FlexActors({node_ids[client_id]: FlexRole.server_client for client_id in range(n_clients_aggregators, n_nodes-rest_clients)})
    )

    actors.update(
        FlexActors({node_ids[client_id]: FlexRole.client for client_id in range(n_clients_aggregators+n_clients_servers, n_nodes)})
    )

    if n_clients_servers == 0:
        actors['server'] = FlexRole.server

    return actors

In [ ]:
custom_actors = custom_architecture(list(federated_iris.keys()))

custom_pool = FlexPool(federated_iris, custom_actors)

clients = custom_pool.clients
aggregators = custom_pool.aggregators
servers = custom_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

Another custom architecture that might be present in a real scenario, is one where we have N clients, M aggregators and a server. In this architecture, we try to simulate that the N clients are distributed by locations, and those aggregators that are near those clients, will communicate each other in order to reduce the connection time during training.

In [ ]:
def real_scenario(node_ids: list, n_aggregators: int) -> FlexActors:
    if "server" in node_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    actors = FlexActors(
        {node_ids[client_id]: FlexRole.client for client_id in node_ids}
    )

    actors.update(
        FlexActors({f"aggregator_{agg_id}": FlexRole.aggregator for agg_id in range(n_aggregators)})
    )

    actors['server'] = FlexRole.server

    return actors

In [ ]:
real_scenario_actors = real_scenario(list(federated_iris.keys()), n_aggregators=2)

real_scenario_pool = FlexPool(federated_iris, real_scenario_actors)

clients = real_scenario_pool.clients
aggregators = real_scenario_pool.aggregators
servers = real_scenario_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

After creating the pool, we can initilize the server model using the `map` function available in the `FlexPool`class, that act as a communication function between the different actors from the same pool. 

In this example, we are going to use the `init_server_model` decorator. To learn more about the `map` function and how to use it with the primitives or the decorators, please, refer to the notebooks that show how to train a model using the different frameworks.

In [ ]:
from flex.pool.decorators import init_server_model

import tensorflow as tf

@init_server_model
def define_model(**kwargs):
    from copy import deepcopy
    input_shape = kwargs['input_shape']
    n_classes = kwargs['n_classes']
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=input_shape))
    model.add(tf.keras.layers.Dense(units=64))
    model.add(tf.keras.layers.Dense(n_classes))
    model.compile(
        optimizer='adam',
        loss=tf.losses.BinaryCrossentropy(from_logits=True),
        metrics=[tf.metrics.Accuracy(name='accuracy')],
    )
    from flex.model import FlexModel
    server_flex_model = FlexModel()
    server_flex_model["model"] = model
    server_flex_model["loss"] = deepcopy(model.loss)
    server_flex_model["metrics"] = deepcopy(model.compiled_metrics._metrics)
    server_flex_model["optimizer"] = deepcopy(model.optimizer)
    return server_flex_model

In [ ]:
real_scenario_pool.servers.map(define_model, input_shape=iris.data.shape[1], n_classes=len(set(iris.target)))

print("Let's show the models available in the pool:")
print(real_scenario_pool._models)

We can see that the server model is initilized, and that this model must be deployed to the clients to start training the model.

Usually, we are going to work with the client-server architecture, as is the most common arquitecture in Federated Learning, so we can just simply use the classmethod available in `FlexPool`. Using the `client_server_architecture`or the `p2p_architecture`functions, we have to pass as argument the function that initilize the server model. 

In [ ]:
pool = FlexPool.client_server_architecture(fed_dataset=federated_iris, init_func=define_model, input_shape=iris.data.shape[1], n_classes=len(set(iris.target)))

clients = pool.clients
aggregators = pool.aggregators
servers = pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

print(f"Server model initialized: {pool.servers._models}")

# END

We have shown how to create custom architetures using the FlexPool initializer instead of using the `class_methods` that are already implemented in the `FlexPool`class.